In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
.builder \
.master("spark://25.39.43.23:7077") \
.appName("Small Data") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/03 17:00:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df = spark.read.csv("hdfs://dmk:9000/SmallData/data1.csv", header=True, inferSchema=True)
df.show()

22/01/03 17:02:13 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (25.39.43.23 executor 2): org.apache.hadoop.hdfs.BlockMissingException: Could not obtain block: BP-1978874256-25.70.124.61-1641015541909:blk_1073741829_1005 file=/SmallData/data1.csv
	at org.apache.hadoop.hdfs.DFSInputStream.refetchLocations(DFSInputStream.java:1007)
	at org.apache.hadoop.hdfs.DFSInputStream.chooseDataNode(DFSInputStream.java:990)
	at org.apache.hadoop.hdfs.DFSInputStream.chooseDataNode(DFSInputStream.java:969)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:677)
	at org.apache.hadoop.hdfs.DFSInputStream.readWithStrategy(DFSInputStream.java:884)
	at org.apache.hadoop.hdfs.DFSInputStream.read(DFSInputStream.java:957)
	at java.base/java.io.DataInputStream.read(DataInputStream.java:149)
	at org.apache.hadoop.mapreduce.lib.input.UncompressedSplitLineReader.fillBuffer(UncompressedSplitLineReader.java:62)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:22

In [4]:
df2 = spark.read.csv("hdfs://dmk:9000/SmallData/data2.csv", header=True, inferSchema=True)
df2.show()

+--------------------+--------+----------+---------+--------------------+--------------------+
|                date|   label|page_index|sub_label|                text|                 url|
+--------------------+--------+----------+---------+--------------------+--------------------+
|13/12/2021 21:11:...|    null|      null|     null|                null|                null|
|                   "|Giáo dục|         1|     null|"Năm 2002, anh Lê...|Thầy Long cũng kh...|
|13/12/2021 21:40:...|    null|      null|     null|                null|                null|
|                   "|Giáo dục|         1|     null|Với mỗi bài thi, ...|https://vietnamne...|
|13/12/2021 21:58:...|    null|      null|     null|                null|                null|
|                   "|Giáo dục|         1|     null|UBND TP.Hà Nội mớ...|https://vietnamne...|
|13/12/2021 21:11:...|    null|      null|     null|                null|                null|
|                   "|Giáo dục|         1|     nul

In [5]:
df3 = spark.read.json("hdfs://dmk:9000/SmallData/data3.json")
df3.show()

+---------------+--------------------+---------+----------+---------------+--------------------+--------------------+
|_corrupt_record|                date|    label|page_index|      sub_label|                text|                 url|
+---------------+--------------------+---------+----------+---------------+--------------------+--------------------+
|              [|                null|     null|      null|           null|                null|                null|
|           null|13/12/2021 22:51:...|Pháp luật|         1|           null|[Ngày 11/12, phiê...|https://vietnamne...|
|           null|13/12/2021 22:19:...|Pháp luật|         1|           null|[Viện kiểm sát nh...|https://vietnamne...|
|           null|13/12/2021 22:16:...|Pháp luật|         1|    Hồ sơ vụ án|[Trước đó, ngày 9...|https://vietnamne...|
|           null|13/12/2021 22:30:...|Pháp luật|         1|           null|[Như vậy, cộng vớ...|https://vietnamne...|
|           null|13/12/2021 22:25:...|Pháp luật|        

In [6]:
df4 = spark.read.json("hdfs://dmk:9000/SmallData/data4.json")
df4.show()

+---------------+--------------------+------------+----------+----------+--------------------+--------------------+
|_corrupt_record|                date|       label|page_index| sub_label|                text|                 url|
+---------------+--------------------+------------+----------+----------+--------------------+--------------------+
|              [|                null|        null|      null|      null|                null|                null|
|           null|13/12/2021 22:24:...|    Sức khỏe|         1|      null|[Công văn được g...|https://vietnamne...|
|           null|13/12/2021 22:39:...|    Sức khỏe|         1|      null|[Chiều ngày 12/12...|https://vietnamne...|
|           null|13/12/2021 21:54:...|    Sức khỏe|         1|      null|[Bộ Y tế vừa có v...|https://vietnamne...|
|           null|13/12/2021 22:14:...|    Sức khỏe|         1|      null|[Các bệnh nhân ...|https://vietnamne...|
|           null|13/12/2021 21:53:...|    Sức khỏe|         1|      null

In [7]:
df3 = df3.drop('_corrupt_record')
df4 = df4.drop('_corrupt_record')

In [8]:
tmp1 = df.union(df2)
tmp2 = df3.union(df4)

In [9]:
from pyspark.sql.functions import col, concat_ws
tmp3 = tmp2.withColumn('text', concat_ws(',', col('text')))
tmp3.printSchema()

root
 |-- date: string (nullable = true)
 |-- label: string (nullable = true)
 |-- page_index: long (nullable = true)
 |-- sub_label: string (nullable = true)
 |-- text: string (nullable = false)
 |-- url: string (nullable = true)



In [10]:
data = tmp1.union(tmp3)
data.printSchema()

root
 |-- date: string (nullable = true)
 |-- label: string (nullable = true)
 |-- page_index: long (nullable = true)
 |-- sub_label: string (nullable = true)
 |-- text: string (nullable = true)
 |-- url: string (nullable = true)



In [11]:
data.show(5)

+--------------------+-------+----------+---------+--------------------+--------------------+
|                date|  label|page_index|sub_label|                text|                 url|
+--------------------+-------+----------+---------+--------------------+--------------------+
|13/12/2021 22:04:...|   null|      null|     null|                null|                null|
|                   "|Thời sự|         1|     null|"Ngày 13/12, Thàn...|            Trong đó|
|                null|   null|         1|     null|Sáng 13/12 theo g...|https://vietnamne...|
|13/12/2021 21:57:...|   null|      null|     null|                null|                null|
|                   "|Thời sự|         1|     null|Ngày 13/12, UBND ...|https://vietnamne...|
+--------------------+-------+----------+---------+--------------------+--------------------+
only showing top 5 rows



In [12]:
test = data
test = test.na.drop()

In [13]:
test.summary().show()

+-------+--------------------+--------+-----------------+--------------------+--------------------+-----------------+
|summary|                date|   label|       page_index|           sub_label|                text|              url|
+-------+--------------------+--------+-----------------+--------------------+--------------------+-----------------+
|  count|              110740|  110740|           110740|              110740|              110740|           110740|
|   mean|                null|    null|618.2430828968755|                null|                 3.5|338.6666666666667|
| stddev|                null|    null|401.9309249034142|                null|  3.5355339059327378|765.0073479544118|
|    min|                   "| Bạn đọc|                1|125 năm ngày sinh...|                    |                 |
|    25%|                null|    null|              288|                null|                 1.0|              3.0|
|    50%|                null|    null|              557

In [14]:
test.show()

+---------------+--------+----------+--------------------+--------------------+--------------------+
|           date|   label|page_index|           sub_label|                text|                 url|
+---------------+--------+----------+--------------------+--------------------+--------------------+
|              "| Thời sự|         1|          Môi trường|Ngày 22/7/2021, B...|https://vietnamne...|
|              "| Thời sự|         1|  An toàn giao thông|Sáng nay 12/12, t...|https://vietnamne...|
|              "| Thời sự|         1|  An toàn giao thông|Chủ tịch UBND h...|https://vietnamne...|
|              "| Thời sự|         1|  An toàn giao thông|"Ngày 13/12, Phó ...| bia - không lái ...|
|              "|Giải trí|         1|        Thế giới sao|"Sau khi chia tay...| hay ít nhất mình...|
|              "|Giải trí|         1|          Thời trang|Ca sĩ , tên thật ...|https://vietnamne...|
|              "|Giải trí|         1|          Thời trang|" diễn ra sáng 13...| ngay từ đầu

## Upload data to ES

In [15]:
import requests, json, os
from elasticsearch import Elasticsearch

In [16]:
res = requests.get('http://localhost:9200')
print(res.content)
es = Elasticsearch([{'host':'localhost', 'port':'9200'}])

b'{\n  "name" : "node-1",\n  "cluster_name" : "small_data",\n  "cluster_uuid" : "knRbiWHBRpCp2_8FE9dbLQ",\n  "version" : {\n    "number" : "7.16.2",\n    "build_flavor" : "default",\n    "build_type" : "deb",\n    "build_hash" : "2b937c44140b6559905130a8650c64dbd0879cfb",\n    "build_date" : "2021-12-18T19:42:46.604893745Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.10.1",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [17]:
data_ES = test.drop(test.url)
data_ES = data_ES.drop(test.text)
data_ES = data_ES.drop(test.date)
data_ES.printSchema()

root
 |-- label: string (nullable = true)
 |-- page_index: long (nullable = true)
 |-- sub_label: string (nullable = true)



In [18]:
data_ES = data_ES.toJSON().collect()

In [22]:
import tqdm

index = 'small_data'
def upload_to_ES(data):
    i = 1
    for item in range(len(data)):
#         try:
        es.index(index='small_data1', doc_type='data', id=i, body=data[item])
        i += 1
#         except:
#             print('Error when uploading to ES')
    

In [23]:
upload_to_ES(data_ES)

/tmp/ipykernel_27746/359917851.py:8: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='small_data1', doc_type='data', id=i, body=data[item])
/home/hadoop/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/home/hadoop/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}